In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 8
tree_depth = 10
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.206886291503906 | KNN Loss: 6.2244954109191895 | BCE Loss: 1.9823912382125854
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.132919311523438 | KNN Loss: 6.224689483642578 | BCE Loss: 1.9082294702529907
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.164770126342773 | KNN Loss: 6.224438190460205 | BCE Loss: 1.9403314590454102
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.090507507324219 | KNN Loss: 6.223604202270508 | BCE Loss: 1.8669028282165527
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.19241714477539 | KNN Loss: 6.222877025604248 | BCE Loss: 1.9695402383804321
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.14727783203125 | KNN Loss: 6.222434997558594 | BCE Loss: 1.9248433113098145
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.140485763549805 | KNN Loss: 6.221712589263916 | BCE Loss: 1.9187731742858887
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.113654136657715 | KNN Loss: 6.220485687255859 | BCE Loss: 1.893168

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 4.986264228820801 | KNN Loss: 3.8754231929779053 | BCE Loss: 1.1108407974243164
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 4.874971389770508 | KNN Loss: 3.7803409099578857 | BCE Loss: 1.094630241394043
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 4.799947738647461 | KNN Loss: 3.687481641769409 | BCE Loss: 1.1124663352966309
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 4.735187530517578 | KNN Loss: 3.6294658184051514 | BCE Loss: 1.1057215929031372
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 4.600740432739258 | KNN Loss: 3.5104541778564453 | BCE Loss: 1.090286135673523
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 4.573747158050537 | KNN Loss: 3.4555721282958984 | BCE Loss: 1.1181750297546387
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 4.487905979156494 | KNN Loss: 3.3704724311828613 | BCE Loss: 1.1174335479736328
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 4.407155990600586 | KNN Loss: 3.301198720932007 | BCE 

Epoch 21 / 500 | iteration 15 / 30 | Total Loss: 3.745793104171753 | KNN Loss: 2.7118005752563477 | BCE Loss: 1.0339925289154053
Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 3.7594337463378906 | KNN Loss: 2.709543466567993 | BCE Loss: 1.049890398979187
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 3.7088677883148193 | KNN Loss: 2.6439218521118164 | BCE Loss: 1.064945936203003
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 3.787175178527832 | KNN Loss: 2.700822591781616 | BCE Loss: 1.0863525867462158
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 3.7421951293945312 | KNN Loss: 2.6866912841796875 | BCE Loss: 1.0555038452148438
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 3.7248144149780273 | KNN Loss: 2.6636526584625244 | BCE Loss: 1.0611616373062134
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 3.732130289077759 | KNN Loss: 2.6874048709869385 | BCE Loss: 1.0447254180908203
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 3.753727674484253 | KNN Loss: 2.7074761390686035 |

Epoch 32 / 500 | iteration 5 / 30 | Total Loss: 3.6425485610961914 | KNN Loss: 2.6196279525756836 | BCE Loss: 1.0229206085205078
Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 3.6593141555786133 | KNN Loss: 2.5833194255828857 | BCE Loss: 1.0759947299957275
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 3.5900542736053467 | KNN Loss: 2.5517096519470215 | BCE Loss: 1.0383446216583252
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 3.6452982425689697 | KNN Loss: 2.5981411933898926 | BCE Loss: 1.0471570491790771
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 3.558393716812134 | KNN Loss: 2.5428829193115234 | BCE Loss: 1.0155107975006104
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 3.5885677337646484 | KNN Loss: 2.533275842666626 | BCE Loss: 1.0552918910980225
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 3.6900157928466797 | KNN Loss: 2.650986671447754 | BCE Loss: 1.0390291213989258
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 3.6802167892456055 | KNN Loss: 2.64267873764038

Epoch 42 / 500 | iteration 25 / 30 | Total Loss: 3.618813991546631 | KNN Loss: 2.572598457336426 | BCE Loss: 1.046215534210205
Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 3.6307477951049805 | KNN Loss: 2.569920539855957 | BCE Loss: 1.0608272552490234
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 3.6177334785461426 | KNN Loss: 2.5625157356262207 | BCE Loss: 1.0552177429199219
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 3.590005874633789 | KNN Loss: 2.553828716278076 | BCE Loss: 1.036177158355713
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 3.61464262008667 | KNN Loss: 2.6170718669891357 | BCE Loss: 0.9975708723068237
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 3.642763614654541 | KNN Loss: 2.5822277069091797 | BCE Loss: 1.0605359077453613
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 3.558879852294922 | KNN Loss: 2.5587778091430664 | BCE Loss: 1.0001020431518555
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 3.5804293155670166 | KNN Loss: 2.5366017818450928 | BCE

...

Epoch 455 / 500 | iteration 5 / 30 | Total Loss: 3.506587505340576 | KNN Loss: 2.482028007507324 | BCE Loss: 1.0245593786239624
Epoch 455 / 500 | iteration 10 / 30 | Total Loss: 3.4664971828460693 | KNN Loss: 2.4481492042541504 | BCE Loss: 1.018347978591919
Epoch 455 / 500 | iteration 15 / 30 | Total Loss: 3.4151315689086914 | KNN Loss: 2.4421021938323975 | BCE Loss: 0.9730292558670044
Epoch 455 / 500 | iteration 20 / 30 | Total Loss: 3.4269938468933105 | KNN Loss: 2.422834634780884 | BCE Loss: 1.0041590929031372
Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 3.4638149738311768 | KNN Loss: 2.476300001144409 | BCE Loss: 0.9875149130821228
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 3.5066330432891846 | KNN Loss: 2.4695498943328857 | BCE Loss: 1.0370831489562988
Epoch 456 / 500 | iteration 5 / 30 | Total Loss: 3.490180730819702 | KNN Loss: 2.455274820327759 | BCE Loss: 1.0349059104919434
Epoch 456 / 500 | iteration 10 / 30 | Total Loss: 3.4882771968841553 | KNN Loss: 2.4746177196

Epoch 465 / 500 | iteration 20 / 30 | Total Loss: 3.490302085876465 | KNN Loss: 2.4632110595703125 | BCE Loss: 1.0270910263061523
Epoch 465 / 500 | iteration 25 / 30 | Total Loss: 3.4911444187164307 | KNN Loss: 2.46443247795105 | BCE Loss: 1.0267119407653809
Epoch 466 / 500 | iteration 0 / 30 | Total Loss: 3.4638800621032715 | KNN Loss: 2.4437785148620605 | BCE Loss: 1.020101547241211
Epoch 466 / 500 | iteration 5 / 30 | Total Loss: 3.459263563156128 | KNN Loss: 2.4353206157684326 | BCE Loss: 1.0239429473876953
Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 3.4581027030944824 | KNN Loss: 2.446424961090088 | BCE Loss: 1.0116777420043945
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 3.4539642333984375 | KNN Loss: 2.453716993331909 | BCE Loss: 1.0002471208572388
Epoch 466 / 500 | iteration 20 / 30 | Total Loss: 3.4721603393554688 | KNN Loss: 2.465996265411377 | BCE Loss: 1.0061639547348022
Epoch 466 / 500 | iteration 25 / 30 | Total Loss: 3.4632041454315186 | KNN Loss: 2.4549984931

Epoch 476 / 500 | iteration 5 / 30 | Total Loss: 3.435896158218384 | KNN Loss: 2.4375128746032715 | BCE Loss: 0.9983832240104675
Epoch 476 / 500 | iteration 10 / 30 | Total Loss: 3.4385604858398438 | KNN Loss: 2.4401745796203613 | BCE Loss: 0.9983858466148376
Epoch 476 / 500 | iteration 15 / 30 | Total Loss: 3.512115240097046 | KNN Loss: 2.4807064533233643 | BCE Loss: 1.0314087867736816
Epoch 476 / 500 | iteration 20 / 30 | Total Loss: 3.46596097946167 | KNN Loss: 2.4419243335723877 | BCE Loss: 1.0240366458892822
Epoch 476 / 500 | iteration 25 / 30 | Total Loss: 3.487907886505127 | KNN Loss: 2.4812941551208496 | BCE Loss: 1.0066137313842773
Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 3.494438648223877 | KNN Loss: 2.4712114334106445 | BCE Loss: 1.0232272148132324
Epoch 477 / 500 | iteration 5 / 30 | Total Loss: 3.5344932079315186 | KNN Loss: 2.517587423324585 | BCE Loss: 1.0169057846069336
Epoch 477 / 500 | iteration 10 / 30 | Total Loss: 3.449920654296875 | KNN Loss: 2.44377946853

Epoch 486 / 500 | iteration 20 / 30 | Total Loss: 3.497659206390381 | KNN Loss: 2.462867498397827 | BCE Loss: 1.0347917079925537
Epoch 486 / 500 | iteration 25 / 30 | Total Loss: 3.446634531021118 | KNN Loss: 2.426318883895874 | BCE Loss: 1.0203156471252441
Epoch 487 / 500 | iteration 0 / 30 | Total Loss: 3.4695658683776855 | KNN Loss: 2.4240596294403076 | BCE Loss: 1.0455061197280884
Epoch 487 / 500 | iteration 5 / 30 | Total Loss: 3.4758872985839844 | KNN Loss: 2.467365026473999 | BCE Loss: 1.0085222721099854
Epoch 487 / 500 | iteration 10 / 30 | Total Loss: 3.442422866821289 | KNN Loss: 2.435115098953247 | BCE Loss: 1.007307767868042
Epoch 487 / 500 | iteration 15 / 30 | Total Loss: 3.4759304523468018 | KNN Loss: 2.4420318603515625 | BCE Loss: 1.0338985919952393
Epoch 487 / 500 | iteration 20 / 30 | Total Loss: 3.4397456645965576 | KNN Loss: 2.422353982925415 | BCE Loss: 1.0173916816711426
Epoch 487 / 500 | iteration 25 / 30 | Total Loss: 3.460203170776367 | KNN Loss: 2.462445497512

Epoch 497 / 500 | iteration 5 / 30 | Total Loss: 3.4742002487182617 | KNN Loss: 2.4685566425323486 | BCE Loss: 1.0056437253952026
Epoch 497 / 500 | iteration 10 / 30 | Total Loss: 3.459146738052368 | KNN Loss: 2.4547905921936035 | BCE Loss: 1.0043561458587646
Epoch 497 / 500 | iteration 15 / 30 | Total Loss: 3.43691349029541 | KNN Loss: 2.4403629302978516 | BCE Loss: 0.996550440788269
Epoch 497 / 500 | iteration 20 / 30 | Total Loss: 3.407719373703003 | KNN Loss: 2.4244635105133057 | BCE Loss: 0.9832558631896973
Epoch 497 / 500 | iteration 25 / 30 | Total Loss: 3.4951319694519043 | KNN Loss: 2.4849376678466797 | BCE Loss: 1.0101943016052246
Epoch 498 / 500 | iteration 0 / 30 | Total Loss: 3.4707562923431396 | KNN Loss: 2.453183650970459 | BCE Loss: 1.0175726413726807
Epoch 498 / 500 | iteration 5 / 30 | Total Loss: 3.4635910987854004 | KNN Loss: 2.4739856719970703 | BCE Loss: 0.9896055459976196
Epoch 498 / 500 | iteration 10 / 30 | Total Loss: 3.4412012100219727 | KNN Loss: 2.442117929

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.8485,  4.5208,  3.0494,  2.1339,  3.6342,  0.9707,  1.8464,  2.4946,
          2.8739,  1.1835,  2.6940,  2.8041,  0.4477,  2.2759,  0.6752,  1.7183,
          1.1334,  3.7789,  2.5588,  0.8831,  1.1705,  1.5609,  2.6076,  2.4545,
          3.0419,  0.7232,  2.4458,  0.8959,  1.3016,  0.3329, -0.1169,  0.7758,
          0.1671, -0.0449,  0.5050,  0.4908,  0.8311,  1.7182,  1.0354,  0.3517,
          0.7787, -0.2847,  0.0153,  1.3845,  2.5655,  0.6838, -0.5005, -0.4219,
          1.6837,  2.9835,  2.2762, -0.1934,  0.7293,  0.9027, -0.5613,  0.6813,
          1.0101,  0.4168,  1.8063,  1.5576, -0.4391,  0.3404,  0.1638,  1.2698,
          1.8243,  0.3636, -1.4577, -0.0348,  2.2464,  2.4268,  1.2849,  0.8021,
          0.1023,  2.4085,  1.5305,  1.6561,  0.4774,  0.2325,  0.6024,  1.7814,
          0.1514,  0.6269,  0.3638, -0.0341,  0.1555, -0.8348, -2.1540, -0.0870,
          0.7430, -2.2588,  0.6612, -0.3409, -0.6863, -0.9409,  0.9034,  1.6642,
         -0.6596, -0.6228,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 77.00it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 002 | Total loss: 9.645 | Reg loss: 0.012 | Tree loss: 9.645 | Accuracy: 0.000000 | 1.038 sec/iter
Epoch: 00 | Batch: 001 / 002 | Total loss: 9.639 | Reg loss: 0.011 | Tree loss: 9.639 | Accuracy: 0.000000 | 0.952 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 01 | Batch: 000 / 002 | Total loss: 9.635 | Reg loss: 0.003 | Tree loss: 9.635 | Accuracy: 0.000000 | 1.088 sec/iter
Epoch: 01 | Batch: 001 / 002 | Total loss: 9.628 | Reg loss: 0.003 | Tree loss: 9.628 | Accuracy: 0.000000 | 1.028 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 

Epoch: 15 | Batch: 000 / 002 | Total loss: 9.578 | Reg loss: 0.004 | Tree loss: 9.578 | Accuracy: 0.000000 | 1.092 sec/iter
Epoch: 15 | Batch: 001 / 002 | Total loss: 9.568 | Reg loss: 0.004 | Tree loss: 9.568 | Accuracy: 0.009390 | 1.084 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 16 | Batch: 000 / 002 | Total loss: 9.572 | Reg loss: 0.004 | Tree loss: 9.572 | Accuracy: 0.001953 | 1.093 sec/iter
Epoch: 16 | Batch: 001 / 002 | Total loss: 9.567 | Reg loss: 0.004 | Tree loss: 9.567 | Accuracy: 0.007042 | 1.086 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 31 | Batch: 000 / 002 | Total loss: 9.514 | Reg loss: 0.006 | Tree loss: 9.514 | Accuracy: 0.269531 | 1.093 sec/iter
Epoch: 31 | Batch: 001 / 002 | Total loss: 9.505 | Reg loss: 0.006 | Tree loss: 9.505 | Accuracy: 0.295775 | 1.09 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 32 | Batch: 000 / 002 | Total loss: 9.510 | Reg loss: 0.006 | Tree loss: 9.510 | Accuracy: 0.259766 | 1.093 sec/iter
Epoch: 32 | Batch: 001 / 002 | Total loss: 9.503 

Epoch: 46 | Batch: 000 / 002 | Total loss: 9.457 | Reg loss: 0.007 | Tree loss: 9.457 | Accuracy: 0.267578 | 1.092 sec/iter
Epoch: 46 | Batch: 001 / 002 | Total loss: 9.446 | Reg loss: 0.007 | Tree loss: 9.446 | Accuracy: 0.298122 | 1.089 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 47 | Batch: 000 / 002 | Total loss: 9.448 | Reg loss: 0.007 | Tree loss: 9.448 | Accuracy: 0.298828 | 1.092 sec/iter
Epoch: 47 | Batch: 001 / 002 | Total loss: 9.448 | Reg loss: 0.007 | Tree loss: 9.448 | Accuracy: 0.260563 | 1.09 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6:

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 62 | Batch: 000 / 002 | Total loss: 9.389 | Reg loss: 0.009 | Tree loss: 9.389 | Accuracy: 0.263672 | 1.092 sec/iter
Epoch: 62 | Batch: 001 / 002 | Total loss: 9.374 | Reg loss: 0.009 | Tree loss: 9.374 | Accuracy: 0.302817 | 1.09 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 63 | Batch: 000 / 002 | Total loss: 9.377 | Reg loss: 0.009 | Tree loss: 9.377 | Accuracy: 0.296875 | 1.092 sec/iter
Epoch: 63 | Batch: 001 / 002 | Total loss: 9.379 

Epoch: 77 | Batch: 000 / 002 | Total loss: 9.308 | Reg loss: 0.010 | Tree loss: 9.308 | Accuracy: 0.273438 | 1.091 sec/iter
Epoch: 77 | Batch: 001 / 002 | Total loss: 9.289 | Reg loss: 0.010 | Tree loss: 9.289 | Accuracy: 0.291080 | 1.09 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 78 | Batch: 000 / 002 | Total loss: 9.301 | Reg loss: 0.010 | Tree loss: 9.301 | Accuracy: 0.263672 | 1.092 sec/iter
Epoch: 78 | Batch: 001 / 002 | Total loss: 9.284 | Reg loss: 0.011 | Tree loss: 9.284 | Accuracy: 0.302817 | 1.09 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 

Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 93 | Batch: 000 / 002 | Total loss: 9.177 | Reg loss: 0.013 | Tree loss: 9.177 | Accuracy: 0.292969 | 1.092 sec/iter
Epoch: 93 | Batch: 001 / 002 | Total loss: 9.166 | Reg loss: 0.013 | Tree loss: 9.166 | Accuracy: 0.267606 | 1.091 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
Epoch: 94 | Batch: 000 / 002 | Total loss: 9.170 | Reg loss: 0.013 | Tree loss: 9.170 | Accuracy: 0.279297 | 1.093 sec/iter
Epoch: 94 | Batch: 001 / 002 | Total loss: 9.154

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 10.0


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 1024


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))



============== Pattern 105 ==============
============== Pattern 106 ==============
============== Pattern 107 ==============
============== Pattern 108 ==============
============== Pattern 109 ==============
============== Pattern 110 ==============
============== Pattern 111 ==============
============== Pattern 112 ==============
============== Pattern 113 ==============
============== Pattern 114 ==============
============== Pattern 115 ==============
============== Pattern 116 ==============
============== Pattern 117 ==============
============== Pattern 118 ==============
============== Pattern 119 ==============
============== Pattern 120 ==============
============== Pattern 121 ==============
============== Pattern 122 ==============
============== Pattern 123 ==============
============== Pattern 124 ==============
============== Pattern 125 ==============
============== Pattern 126 ==============
============== Pattern 127 ==============
============== Pattern 128 ======

============== Pattern 339 ==============
============== Pattern 340 ==============
============== Pattern 341 ==============
============== Pattern 342 ==============
============== Pattern 343 ==============
============== Pattern 344 ==============
============== Pattern 345 ==============
============== Pattern 346 ==============
============== Pattern 347 ==============
============== Pattern 348 ==============
============== Pattern 349 ==============
============== Pattern 350 ==============
============== Pattern 351 ==============
============== Pattern 352 ==============
============== Pattern 353 ==============
============== Pattern 354 ==============
============== Pattern 355 ==============
============== Pattern 356 ==============
============== Pattern 357 ==============
============== Pattern 358 ==============
============== Pattern 359 ==============
============== Pattern 360 ==============
============== Pattern 361 ==============
============== Pattern 362 =======

============== Pattern 558 ==============
============== Pattern 559 ==============
============== Pattern 560 ==============
============== Pattern 561 ==============
============== Pattern 562 ==============
============== Pattern 563 ==============
============== Pattern 564 ==============
============== Pattern 565 ==============
============== Pattern 566 ==============
============== Pattern 567 ==============
============== Pattern 568 ==============
============== Pattern 569 ==============
============== Pattern 570 ==============
============== Pattern 571 ==============
============== Pattern 572 ==============
============== Pattern 573 ==============
============== Pattern 574 ==============
============== Pattern 575 ==============
============== Pattern 576 ==============
============== Pattern 577 ==============
============== Pattern 578 ==============
============== Pattern 579 ==============
============== Pattern 580 ==============
============== Pattern 581 =======

============== Pattern 753 ==============
============== Pattern 754 ==============
============== Pattern 755 ==============
============== Pattern 756 ==============
============== Pattern 757 ==============
============== Pattern 758 ==============
============== Pattern 759 ==============
============== Pattern 760 ==============
============== Pattern 761 ==============
============== Pattern 762 ==============
============== Pattern 763 ==============
============== Pattern 764 ==============
============== Pattern 765 ==============
============== Pattern 766 ==============
============== Pattern 767 ==============
============== Pattern 768 ==============
============== Pattern 769 ==============
============== Pattern 770 ==============
============== Pattern 771 ==============
============== Pattern 772 ==============
============== Pattern 773 ==============
============== Pattern 774 ==============
============== Pattern 775 ==============
============== Pattern 776 =======